In [1]:
import requests
import pandas as pd
import matplotlib.pyplot as plt

base_url = "https://api.nhtsa.gov/SafetyRatings"
year = 2022

car_brands = ['Ford', 'Toyota', 'Tesla', 'Chevrolet', 'Honda', 'Nissan', 'Jeep', 'Hyundai', 'Subaru', 'GMC', 'Ram', 'Volkswagen', 'BMW', 'Mercedes-Benz', 'Dodge', 'Lexus', 'Audi', 'Mazda', 'Kia', 'Buick', 'Cadillac']


In [2]:

data = {}

for make in car_brands:
    # Fetch all models for the current make and year
    models_url = f"{base_url}/modelyear/{year}/make/{make}"
    response = requests.get(models_url)
    models_data = response.json()
    
    # Extract the list of models for the current make
    models = [model['Model'] for model in models_data['Results']]
    
    for model in models:
        # Fetch safety ratings for the current model
        safety_ratings_url = f"{base_url}/modelyear/{year}/make/{make}/model/{model}"
        response = requests.get(safety_ratings_url)
        safety_data = response.json()
        
        # Store the safety ratings for the current model
        data[f"{make} {model}"] = safety_data['Results']


In [3]:
# Flatten the nested dictionary structure
flattened_data = []
for make_model, attributes in data.items():
    for item in attributes:
        item['Make_Model'] = make_model
        flattened_data.append(item)

# Convert the flattened data to a DataFrame
df = pd.DataFrame(flattened_data)

In [4]:

alt_url = "https://api.nhtsa.gov/SafetyRatings/VehicleId"

# Fetch overall ratings for each vehicle and add them to the DataFrame
for index, row in df.iterrows():
    vehicle_id = row['VehicleId']
    response = requests.get(f"{alt_url}/{vehicle_id}")
    overall_rating_data = response.json()
    overall_rating = overall_rating_data['Results'][0]['OverallRating']
    df.at[index, 'OverallRating'] = overall_rating


In [5]:

# Save the DataFrame with all data including overall ratings to a CSV file
output_csv_path = 'complete_vehicle_safety_ratings.csv'
df.to_csv(output_csv_path, index=False)


Index(['VehicleDescription', 'VehicleId', 'Make_Model', 'OverallRating'], dtype='object')


In [6]:
import requests
import pandas as pd

def fetch_nhtsa_recall_data_for_year(year):
    # Base URL for the NHTSA API for recall data
    rec_url = "https://api.nhtsa.gov/products/vehicle/recalls"

    # Define the parameters for the API request
    params = {
        'modelYear': year,
    }

    # Make the API request
    response = requests.get(rec_url, params=params)

    # Check for a successful response
    if response.status_code == 200:
        # Convert the JSON response to a DataFrame
        return pd.DataFrame(response.json()['results'])
    else:
        print(f"Failed to fetch data for year {year}: Status code {response.status_code}")
        return None



In [7]:
# Example usage: Fetch recall data for the year 2022
year = 2022
recall_data = fetch_nhtsa_recall_data_for_year(year)

# Check if data was fetched and display the first few rows
if recall_data is not None:
    print(recall_data.head())
else:
    print(f"No data fetched for year {year}.")


Failed to fetch data for year 2022: Status code 403
No data fetched for year 2022.


In [10]:
import requests
import pandas as pd

# Base URLs
base_url_safety_ratings = "https://api.nhtsa.gov/SafetyRatings"
base_url_recalls = "https://api.nhtsa.gov/recalls/recallsByVehicle"
year = 2022

car_brands = ['Ford', 'Toyota', 'Tesla', 'Chevrolet', 'Honda', 'Nissan', 'Jeep', 'Hyundai', 'Subaru', 'GMC', 'Ram', 'Volkswagen', 'BMW', 'Mercedes-Benz', 'Dodge', 'Lexus', 'Audi', 'Mazda', 'Kia', 'Buick', 'Cadillac', 'Lincoln']

# Data structures for storing results
safety_ratings_data = []
recall_data = []

for make in car_brands:
    # Fetch all models for the current make and year for safety ratings
    models_url = f"{base_url_safety_ratings}/modelyear/{year}/make/{make}"
    response = requests.get(models_url)
    models_data = response.json()

    # Extract the list of models for the current make
    models = [model['Model'] for model in models_data['Results']]

    for model in models:
        # Fetch safety ratings for the current model
        safety_ratings_url = f"{base_url_safety_ratings}/modelyear/{year}/make/{make}/model/{model}"
        response = requests.get(safety_ratings_url)
        safety_data = response.json()
        
        # Store the safety ratings for the current model
        for item in safety_data['Results']:
            item['Make'] = make
            item['Model'] = model
            safety_ratings_data.append(item)

        # Fetch recall data for the current make and model
        recall_response = requests.get(base_url_recalls, params={'make': make, 'model': model, 'modelYear': year})
        if recall_response.status_code == 200:
            recall_results = recall_response.json()['results']
            for recall in recall_results:
                recall['Make'] = make
                recall['Model'] = model
                recall_data.append(recall)

# Convert the data to DataFrames
df_safety_ratings = pd.DataFrame(safety_ratings_data)
df_recall_data = pd.DataFrame(recall_data)

# Displaying the first few rows of each DataFrame
print("Safety Ratings Data:")
print(df_safety_ratings.head())

print("\nRecall Data:")
print(df_recall_data.head())



Safety Ratings Data:
               VehicleDescription  VehicleId  Make         Model
0    2022 Ford Bronco 2DR SUV 4WD      17037  Ford        BRONCO
1    2022 Ford Bronco 4DR SUV 4WD      16331  Ford        BRONCO
2  2022 Ford Bronco Sport SUV 4WD      16332  Ford  BRONCO SPORT
3      2022 Ford EcoSport SUV AWD      16376  Ford      ECOSPORT
4      2022 Ford EcoSport SUV FWD      16333  Ford      ECOSPORT

Recall Data:
         Manufacturer NHTSACampaignNumber parkIt parkOutSide  \
0  Ford Motor Company           22V411000  False       False   
1  Ford Motor Company           22V451000  False       False   
2  Ford Motor Company           22V825000  False       False   
3  Ford Motor Company           22V928000  False       False   
4  Ford Motor Company           23V070000  False       False   

  ReportReceivedDate                                     Component  \
0         10/06/2022             LATCHES/LOCKS/LINKAGES:DOORS:LOCK   
1         24/06/2022                         VISIB

In [11]:
df_safety_recall = df_recall_data.merge(df_safety_ratings)

In [12]:
merged_df = pd.merge(df_safety_recall, df, on='VehicleId', how='inner')


In [13]:
print(df.head)

<bound method NDFrame.head of                  VehicleDescription  VehicleId         Make_Model  \
0      2022 Ford Bronco 2DR SUV 4WD      17037        Ford BRONCO   
1      2022 Ford Bronco 4DR SUV 4WD      16331        Ford BRONCO   
2    2022 Ford Bronco Sport SUV 4WD      16332  Ford BRONCO SPORT   
3        2022 Ford EcoSport SUV AWD      16376      Ford ECOSPORT   
4        2022 Ford EcoSport SUV FWD      16333      Ford ECOSPORT   
..                              ...        ...                ...   
649       2022 Cadillac XT4 SUV FWD      16077       Cadillac XT4   
650       2022 Cadillac XT5 SUV AWD      16086       Cadillac XT5   
651       2022 Cadillac XT5 SUV FWD      16076       Cadillac XT5   
652       2022 Cadillac XT6 SUV AWD      16088       Cadillac XT6   
653       2022 Cadillac XT6 SUV FWD      16078       Cadillac XT6   

    OverallRating  
0       Not Rated  
1       Not Rated  
2               5  
3               4  
4               4  
..            ...  
6

use csvs

In [14]:
merged_df.to_csv("vehicle_info.csv")

In [15]:
merged_df.to_json("vehicle_info.json")